In [1]:
import pandas as pd
from cuotasimss import CuotasImss
from imss import CalculoIMSSObrero, CalculoIMSSPatron
from isr import ISR
from datetime import datetime
from sbc import SBC

In [2]:
df = pd.read_excel('./data/plantilla_laboral2.xlsx', sheet_name='Hoja1')
df.head()

,Id,Nombre,Sueldo mensual,Vales,Fecha Ingreso
0,1,Martha Rosa Alvarez Martínez,33256.68,1500.0,2019-06-03
1,2,Tania Monserrat Bañuelos Murillo,25931.50,1297.0,2019-06-17
2,6,Karla Berenice Cano Garcia,22960.82,1148.0,2019-07-23
3,7,Jonathan Covarrubias Sustaita,21719.58,1086.0,2019-03-04
4,8,Miguel Dionicio Luna,27315.62,1366.0,2018-01-09


In [3]:
df['Fecha Ingreso'] = pd.to_datetime(df['Fecha Ingreso'], format='%Y-%m-%d')

In [7]:
date_2022 = pd.to_datetime('2022/12/31')

In [8]:
df['SDI_2022'] = df.apply(lambda x: SBC.funcion_sdi(sueldo = x['Sueldo mensual'], a = x['Fecha Ingreso'], b = date_2022), axis=1)
df['SBC_2022'] = df.apply(lambda x: SBC.funcion_sbc(sueldo = x['Sueldo mensual'], vales = x['Vales'], a = x['Fecha Ingreso'], b = date_2022), axis=1)
df['ISR_2022'] = (df['SDI_2022'] * 30.4).apply(lambda x: ISR.function_isr(x))
df['IMSS_O_2022'] = df['SBC_2022'].apply(lambda x: CalculoIMSSObrero.function_imss_obrero(x))
df['IMSS_P_2022'] = df['SBC_2022'].apply(lambda x: CalculoIMSSPatron.function_imss_patronal(x))
df['ISN_2022'] = df['Sueldo mensual'] * 0.03
_isr_ = df['Sueldo mensual'].apply(lambda x: ISR.function_isr(x))
df['Neto'] = df['Sueldo mensual'] - _isr_ - df['IMSS_O_2022']
df['Aguinaldo_2022'] = df.apply(lambda x: SBC.funcion_aguinaldo(sueldo = x['Sueldo mensual'], a = x['Fecha Ingreso'], b = date_2022), axis=1)
df['Prima Vacacional_2022'] = df.apply(lambda x: SBC.funcion_pv(sueldo = x['Sueldo mensual'], date_ingreso = x['Fecha Ingreso'], date_base = date_2022), axis=1)

In [9]:
df.tail()

,Id,Nombre,Sueldo mensual,Vales,Fecha Ingreso,SDI_2022,SBC_2022,ISR_2022,IMSS_O_2022,IMSS_P_2022,ISN_2022,Neto,Aguinaldo_2022,Prima Vacacional_2022
47,63,Brenda Hernandez Jimenez,17687.64,884.0,2022-06-16,616.238000,616.238000,2560.34,484.76,3552.56,530.6292,14865.96,4775.66,0.0
48,64,Katya Del Rocío López Ramírez,17687.64,884.0,2022-06-16,616.238000,616.238000,2560.34,484.76,3552.56,530.6292,14865.96,4775.66,0.0
49,65,José Alfredo López Madrigal,17687.64,884.0,2022-06-16,616.238000,616.238000,2560.34,484.76,3552.56,530.6292,14865.96,4775.66,0.0
50,66,Jesus Geovanni Ruvalcaba Mota,17687.64,884.0,2022-07-01,616.238000,616.238000,2560.34,484.76,3552.56,530.6292,14865.96,4421.91,0.0
51,67,Luis Manuel Guerrero Alcantara,19500.08,975.0,2022-08-22,679.382667,679.382667,2970.37,538.03,3865.33,585.0024,16238.00,3510.01,0.0


In [17]:
aumento_2023 = 1.09

In [18]:
def vales_2023(vale):
    if (vale * aumento_2023) >= 1500:
        return 1500.0
    else:
        return (vale * aumento_2023)

In [19]:
def imss_pt_2023(x):
    nomina = CalculoIMSSPatron()
    nomina.set_sdi(x)
    nomina._uma = 96.22 * aumento_2023
    return nomina.total_imss()

In [20]:
def imss_ob_2023(x):
    nomina = CalculoIMSSObrero()
    nomina.set_sdi(x)
    nomina._uma = 96.22 * aumento
    return nomina.total_imss()

In [21]:
date_2023 = pd.to_datetime('2023/12/31')

In [22]:
df['Sueldo 2023'] = df['Sueldo mensual'].apply(lambda x: x * aumento_2023)
df['Vales 2023'] = df['Vales'].apply(lambda x: vales_2023(x) )

In [23]:
df['SDI_2023'] = df.apply(lambda x: SBC.funcion_sdi(sueldo = x['Sueldo 2023'], a = x['Fecha Ingreso'], b = date_2023), axis=1)
df['SBC_2023'] = df.apply(lambda x: SBC.funcion_sbc(sueldo = x['Sueldo 2023'], vales = x['Vales 2023'], a = x['Fecha Ingreso'], b = date_2023), axis=1)
df['ISR_2023'] = (df['SDI_2023'] * 30.4).apply(lambda x: ISR.function_isr(x))
df['IMSS_O_2023'] = df['SBC_2023'].apply(lambda x: imss_ob_2023(x))
df['IMSS_P_2023'] = df['SBC_2023'].apply(lambda x: imss_pt_2023(x))
df['ISN_2023'] = df['Sueldo 2023'] * 0.03
_isr_ = df['Sueldo 2023'].apply(lambda x: ISR.function_isr(x))
df['Neto 2023'] = df['Sueldo 2023'] - _isr_ - df['IMSS_O_2023']
df['Aguinaldo_2023'] = df.apply(lambda x: SBC.funcion_aguinaldo(sueldo = x['Sueldo 2023'], a = x['Fecha Ingreso'], b = date_2023), axis=1)
df['Prima Vacacional_2023'] = df.apply(lambda x: SBC.funcion_pv(sueldo = x['Sueldo 2023'], date_ingreso = x['Fecha Ingreso'], date_base = date_2023), axis=1)

In [24]:
df.tail()

,Id,Nombre,Sueldo mensual,Vales,Fecha Ingreso,SDI_2022,SBC_2022,ISR_2022,IMSS_O_2022,IMSS_P_2022,...,Vales 2023,SDI_2023,SBC_2023,ISR_2023,IMSS_O_2023,IMSS_P_2023,ISN_2023,Neto 2023,Aguinaldo_2023,Prima Vacacional_2023
47,63,Brenda Hernandez Jimenez,17687.64,884.0,2022-06-16,616.238000,616.238000,2560.34,484.76,3552.56,...,963.56,671.700920,671.700920,2920.49,528.39,3872.31,578.385828,16074.1976,9639.76,963.98
48,64,Katya Del Rocío López Ramírez,17687.64,884.0,2022-06-16,616.238000,616.238000,2560.34,484.76,3552.56,...,963.56,671.700920,671.700920,2920.49,528.39,3872.31,578.385828,16074.1976,9639.76,963.98
49,65,José Alfredo López Madrigal,17687.64,884.0,2022-06-16,616.238000,616.238000,2560.34,484.76,3552.56,...,963.56,671.700920,671.700920,2920.49,528.39,3872.31,578.385828,16074.1976,9639.76,963.98
50,66,Jesus Geovanni Ruvalcaba Mota,17687.64,884.0,2022-07-01,616.238000,616.238000,2560.34,484.76,3552.56,...,963.56,671.700920,671.700920,2920.49,528.39,3872.31,578.385828,16074.1976,9639.76,963.98
51,67,Luis Manuel Guerrero Alcantara,19500.08,975.0,2022-08-22,679.382667,679.382667,2970.37,538.03,3865.33,...,1062.75,740.532907,740.532907,3367.44,586.45,4213.25,637.652616,17569.7172,10627.54,1062.75
